# AnglE-optimized Text Embeddings

> It is Angle 📐, not Angel 👼.

🔥 基于 AnglE 开箱即用的文本向量库，支持中英双语，可用于文本相似度计算、检索召回、匹配等场景。代码基于 🤗transformers 构建，提供易用的微调接口，可在 3090Ti、 4090 等消费级 GPU 上微调 LLaMA-7B 模型，支持多卡分布式训练。


<a href="https://arxiv.org/abs/2309.12871">
    <img src="https://img.shields.io/badge/Arxiv-2306.06843-yellow.svg?style=flat-square" alt="https://arxiv.org/abs/2309.12871" />
</a>
<a href="https://pypi.org/project/angle_emb/">
    <img src="https://img.shields.io/pypi/v/angle_emb?style=flat-square" alt="PyPI version" />
</a>

<a href="https://pypi.org/project/angle_emb/">
    <img src="https://img.shields.io/pypi/dm/angle_emb?style=flat-square" alt="PyPI Downloads" />
</a>
<a href="http://makeapullrequest.com">
    <img src="https://img.shields.io/badge/PRs-welcome-brightgreen.svg?style=flat-square" alt="http://makeapullrequest.com" />
</a>

[![PWC](https://img.shields.io/endpoint.svg?url=https://paperswithcode.com/badge/angle-optimized-text-embeddings/semantic-textual-similarity-on-sick-r-1)](https://paperswithcode.com/sota/semantic-textual-similarity-on-sick-r-1?p=angle-optimized-text-embeddings)
[![PWC](https://img.shields.io/endpoint.svg?url=https://paperswithcode.com/badge/angle-optimized-text-embeddings/semantic-textual-similarity-on-sts16)](https://paperswithcode.com/sota/semantic-textual-similarity-on-sts16?p=angle-optimized-text-embeddings)
[![PWC](https://img.shields.io/endpoint.svg?url=https://paperswithcode.com/badge/angle-optimized-text-embeddings/semantic-textual-similarity-on-sts15)](https://paperswithcode.com/sota/semantic-textual-similarity-on-sts15?p=angle-optimized-text-embeddings)
[![PWC](https://img.shields.io/endpoint.svg?url=https://paperswithcode.com/badge/angle-optimized-text-embeddings/semantic-textual-similarity-on-sts14)](https://paperswithcode.com/sota/semantic-textual-similarity-on-sts14?p=angle-optimized-text-embeddings)
[![PWC](https://img.shields.io/endpoint.svg?url=https://paperswithcode.com/badge/angle-optimized-text-embeddings/semantic-textual-similarity-on-sts13)](https://paperswithcode.com/sota/semantic-textual-similarity-on-sts13?p=angle-optimized-text-embeddings)
[![PWC](https://img.shields.io/endpoint.svg?url=https://paperswithcode.com/badge/angle-optimized-text-embeddings/semantic-textual-similarity-on-sts12)](https://paperswithcode.com/sota/semantic-textual-similarity-on-sts12?p=angle-optimized-text-embeddings)
[![PWC](https://img.shields.io/endpoint.svg?url=https://paperswithcode.com/badge/angle-optimized-text-embeddings/semantic-textual-similarity-on-sts-benchmark)](https://paperswithcode.com/sota/semantic-textual-similarity-on-sts-benchmark?p=angle-optimized-text-embeddings)



如果你有使用我们的代码及预训练模型，欢迎给我们三连，三连方式为：
1) 给本项目 GitHub 加个 star
2) 粘贴以下引用信息到你 paper 的 bibtex
3) 在你的 paper 正文中引用

```bibtex
@article{li2023angle,
  title={AnglE-Optimized Text Embeddings},
  author={Li, Xianming and Li, Jing},
  journal={arXiv preprint arXiv:2309.12871},
  year={2023}
}
```

# 1. 安装依赖

In [2]:
!pip install -U angle-emb

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 7.7 MB 23.4 MB/s            
  Using cached bitsandbytes-0.41.1-py3-none-any.whl (92.6 MB)
     |████████████████████████████████| 3.8 MB 70.0 MB/s            
     |████████████████████████████████| 301 kB 76.6 MB/s            
  Using cached huggingface_hub-0.17.3-py3-none-any.whl (295 kB)


  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.15.1
    Uninstalling huggingface-hub-0.15.1:
      Successfully uninstalled huggingface-hub-0.15.1
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.29.2
    Uninstalling transformers-4.29.2:
      Successfully uninstalled transformers-4.29.2
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [1]:
import os
import random
import numpy as np
import torch


os.environ['CUDA_VISIBLE_DEVICES'] = '0'

seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# 2. 加载数据

需要封装成 datasets.Dataset 格式，必须包含 `text1`, `text2`, `label` 三列，`label` 列是数值类型。

In [2]:
from datasets import load_dataset

ds = load_dataset('shibing624/nli_zh', 'LCQMC')
ds = ds.rename_column('sentence1', 'text1')
ds = ds.rename_column('sentence2', 'text2')
ds = ds.select_columns(["text1", "text2", "label"])

Found cached dataset nli_zh (/home/jupyter-sean/.cache/huggingface/datasets/shibing624___nli_zh/LCQMC/1.0.0/65b555276ee420c801e1c9eb830db959e37f42fa60c68c8b07a4448b8c436706)


  0%|          | 0/3 [00:00<?, ?it/s]

# 3. 加载模型训练

参数主要调整 loss_kwargs，请大家搜索参数，各参数含义参照 Paper: https://arxiv.org/abs/2309.12871


In [3]:
from angle_emb import AnglE, AngleDataTokenizer

angle = AnglE.from_pretrained('hfl/chinese-roberta-wwm-ext', max_length=128, pooling_strategy='cls').cuda()

train_ds = ds['train'].shuffle().map(AngleDataTokenizer(angle.tokenizer, angle.max_length), num_proc=8)
valid_ds = ds['validation'].map(AngleDataTokenizer(angle.tokenizer, angle.max_length), num_proc=8)
test_ds = ds['test'].map(AngleDataTokenizer(angle.tokenizer, angle.max_length), num_proc=8)

Loading cached shuffled indices for dataset at /home/jupyter-sean/.cache/huggingface/datasets/shibing624___nli_zh/LCQMC/1.0.0/65b555276ee420c801e1c9eb830db959e37f42fa60c68c8b07a4448b8c436706/cache-a66a72eedc729b1f.arrow
Loading cached processed dataset at /home/jupyter-sean/.cache/huggingface/datasets/shibing624___nli_zh/LCQMC/1.0.0/65b555276ee420c801e1c9eb830db959e37f42fa60c68c8b07a4448b8c436706/cache-91761d1f760e05a6_*_of_00008.arrow
Loading cached processed dataset at /home/jupyter-sean/.cache/huggingface/datasets/shibing624___nli_zh/LCQMC/1.0.0/65b555276ee420c801e1c9eb830db959e37f42fa60c68c8b07a4448b8c436706/cache-b5b8015b8deaf94e_*_of_00008.arrow
Loading cached processed dataset at /home/jupyter-sean/.cache/huggingface/datasets/shibing624___nli_zh/LCQMC/1.0.0/65b555276ee420c801e1c9eb830db959e37f42fa60c68c8b07a4448b8c436706/cache-312689df2405ddc2_*_of_00008.arrow


In [4]:
angle.fit(
    train_ds=train_ds,
    valid_ds=test_ds,
    output_dir='ckpts/lcqmc',
    batch_size=64,
    epochs=5,
    learning_rate=3e-5,
    save_steps=1000,
    eval_steps=1000,
    warmup_steps=0,
    gradient_accumulation_steps=1,
    loss_kwargs={
        'w1': 1.0,
        'w2': 2.0,
        'w3': 1.0,
        'cosine_tau': 20,
        'ibn_tau': 20,
        'angle_tau': 1.0
    },
    fp16=True,
    logging_steps=500
)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,12.351300
1000,12.176900
1500,12.057100
2000,11.983600
2500,11.902100
3000,11.874400
3500,11.867200
4000,11.461300
4500,11.253400
5000,11.370500


Evaluate: 196it [00:05, 36.50it/s]


new best corrcoef!
save to ckpts/lcqmc/best-checkpoint
corrcoef: 0.7911090709598512, accuracy: 0.8864, best corrcoef: 0.7911090709598512


Evaluate: 196it [00:05, 36.57it/s]


...

corrcoef: 0.7951671579375805, accuracy: 0.89096, best corrcoef: 0.7952815342631034


Evaluate: 196it [00:05, 35.88it/s]


corrcoef: 0.793734826734803, accuracy: 0.8928, best corrcoef: 0.7952815342631034


Evaluate: 196it [00:05, 36.66it/s]

corrcoef: 0.7932197231307199, accuracy: 0.89288, best corrcoef: 0.7952815342631034


# 4. 评估

In [ ]:
# load best checkpoint and evaluate
# 79.52
angle = AnglE.from_pretrained('hfl/chinese-roberta-wwm-ext', pretrained_model_path='ckpts/atec/best-checkpoint').cuda()
angle.evaluate(test_ds, device=angle.device)